In [3]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

import time
import datetime

# 1. Recopilación de fuentes y categorización

In [1]:
# #rutas (ALEJANDO - PERSONAL)

# # Ruta base donde se encuentran los archivos shapefile
# base_path = r"C:\Users\Alejandro\Downloads\DATA\SHP"

# output_path_excel = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Diccionario_2020.xlsx"
# output_path_gpkg = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

#rutas (ALEJANDO - SSD)

# Ruta base donde se encuentran los archivos shapefile
base_path = r"Z:\VOCES\DATA\SHP"

output_path_excel = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Diccionario_2020.xlsx"
output_path_gpkg = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

In [4]:
#1.2 Carga de capas de información puntual, de ambos años, en Jupyter

# Lista para almacenar los GeoDataFrames de cada estado
gdfs = []

# Recorrer los números de estado (del 01 al 32)
for i in range(1, 33):
    # Formatear el número del estado como 'XX' (ejemplo: 01, 02, 03...32)
    state_code = f"{i:02d}"
    # Crear la ruta completa del archivo shapefile
    file_path = os.path.join(base_path, f"{state_code}sip.shp")
    
    # Cargar el shapefile y añadirlo a la lista
    gdf_state = gpd.read_file(file_path)
    gdfs.append(gdf_state)

# Unir todos los GeoDataFrames en uno solo
gdf_2020 = gpd.pd.concat(gdfs, ignore_index=True)

# Verificar la unión exitosa
print(f"Total de registros en el archivo unido: {len(gdf_2020)}")

#guardar en gpkg
gdf_2020.to_file(output_path_gpkg, layer='Eq_full_2020')



#1.3 Desplegado de campos únicos

# Agrupar por 'GEOGRÁFICO' y obtener los valores únicos de 'TIPO' para cada uno
diccionario_geografico_tipo = {}

# Recorrer cada valor único de GEOGRAFICO
for geografico in gdf_2020['GEOGRAFICO'].unique():
    # Filtrar los datos correspondientes al valor de 'GEOGRAFICO' actual
    subset = gdf_2020[gdf_2020['GEOGRAFICO'] == geografico]
    
    # Obtener los valores únicos de 'TIPO' asociados a este 'GEOGRAFICO'
    tipos = subset['TIPO'].unique()
    
    # Añadir al diccionario, si no tiene subniveles, ponemos una lista vacía
    diccionario_geografico_tipo[geografico] = list(tipos) if len(tipos) > 0 else []

# Mostrar el diccionario creado
for geo, tipos in diccionario_geografico_tipo.items():
    print(f"{geo}: {tipos}")


#1.4 Desplegado de valores únicos en campos X e Y
#1.5 Exportación de ambas listas en .xlsx

# Convertir el diccionario en un DataFrame para exportar a Excel
data = []

# Recorrer el diccionario y estructurarlo en filas
for geografico, tipos in diccionario_geografico_tipo.items():
    if tipos:  # Si hay subniveles en 'TIPO'
        for tipo in tipos:
            data.append([geografico, tipo])
    else:
        data.append([geografico, None])  # Si no hay subniveles, ponemos 'None' en 'TIPO'

# Crear DataFrame
df_diccionario = pd.DataFrame(data, columns=['GEOGRAFICO', 'TIPO'])

# Exportar a archivo Excel
df_diccionario.to_excel(output_path_excel, index=False)

print(f"Diccionario exportado exitosamente a {output_path_excel}")


Total de registros en el archivo unido: 440164
Centro de Asistencia Médica: ['Centro de Salud', 'Hospital', 'Centro de Rehabilitación']
Escuela: ['Primaria', 'Preescolar', 'Secundaria', 'Medio Superior', 'Superior', 'Mixto']
Templo: ['No Aplica']
Cementerio: ['No Aplica']
Pozo: ['Agua', 'Petróleo', 'Gas']
Instalación Deportiva o Recreativa: ['Cancha', 'Parque', 'Unidad Deportiva', 'Jardín', 'Lienzo Charro', 'Áreas Verdes', 'Estadio', 'Plaza de Toros', 'Campo de Golf', 'Velódromo', 'Balneario', 'Hipódromo', 'Galgódromo', 'Zoológico', 'Alberca Olímpica', 'Campo de Tiro', 'Autódromo', 'Acuario']
Tanque de Agua: ['Tanque Elevado', 'Caja de Agua']
Instalación Gubernamental: ['Palacio de Gobierno', 'Palacio Municipal']
Mercado: ['No Aplica']
Plaza: ['No Aplica']
Instalación Diversa: ['Monumento u Obelisco', 'Silo', 'Planta de Tratamiento de Agua', 'Gasolinera', 'Museo', 'Centro de Investigación', 'Teatro', 'Edificación Cultural', 'Centro de Espectáculos', 'Cine', 'Estación de Gas', 'Observat

# 2. Ponderación y niveles de servicio

In [5]:
# #rutas (ALEJANDRO - PERSONAL)
# equipamientos_gpkg = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"
# Diccionario_comentado = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Diccionario_pond_2020.xlsx"

#rutas (ALEJANDRO - SSD)
equipamientos_gpkg = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"
Diccionario_comentado = r"Z:\VOCES\DATA\\Tratados\NAC\EQUIPAMIENTOS\Diccionario_pond_2020.xlsx"

In [6]:
# Cargar el diccionario actualizado con las nuevas columnas
diccionario = pd.read_excel(Diccionario_comentado)

# Unir el diccionario con el geodataframe gdf_2020 (asegurarse de que haya un campo en común, por ejemplo 'GEOGRAFICO' y 'TIPO')
gdf_2020 = gdf_2020.merge(diccionario, on=['GEOGRAFICO', 'TIPO'], how='left')

# Filtrar por categorías y exportar a archivos individuales

# Crear una lista de categorías
categorias = ['Educación', 'Salud', 'Recreación', 'Servicios']

# Crear y exportar un GeoDataFrame por cada categoría a una capa dentro del geopackage
for categoria in categorias:
    # Filtrar los equipamientos de la categoría actual
    gdf_categoria = gdf_2020[gdf_2020['CATEGORIA'] == categoria]
    
    # Exportar la categoría como una nueva capa en el geopackage
    gdf_categoria.to_file(equipamientos_gpkg, layer=categoria, driver="GPKG")
    
    print(f"Exportado: {categoria} a la capa {categoria} en el geopackage")

Exportado: Educación a la capa Educación en el geopackage
Exportado: Salud a la capa Salud en el geopackage
Exportado: Recreación a la capa Recreación en el geopackage
Exportado: Servicios a la capa Servicios en el geopackage


# 3. Trabajo a nivel manzana

In [4]:
# # #rutas (ALEJANDRO - PERSONAL)
# manzanas_gpkg_path = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\MZ\Manzanas_NAC.gpkg"
# equipamientos_gpkg = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

#rutas (ALEJANDRO - SSD)
manzanas_gpkg_path = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC.gpkg"
equipamientos_gpkg = r"Z:\VOCES\DATA\Tratados\NAC\EQUIPAMIENTOS\Equipamientos_NAC_2020.gpkg"

In [5]:
#3.1: Cargar el archivo de manzanas
print(f'1 - {datetime.datetime.now().time()}')

# Cargar el archivo de manzanas
gdf_manzanas = gpd.read_file(manzanas_gpkg_path, layer='Mzn_SubTipologias_ue')
print(f'2 - {datetime.datetime.now().time()}')

1 - 13:36:02.006638
2 - 15:12:49.034304


In [8]:
gdf_manzanas

,CVEGEO,POPTOT2010,POPTOT2020,VIVTOT2010,VIVTOT2020,TVIVHAB2010,TVIVHAB2020,TVIVPAR2010,TVIVPAR2020,TVIVPARHAB2010,...,DENS_UEC20,DENS_UEC10,R_TAM,SUBTIPOLOGÍA_ue,geometry,buffer_2m,num_edu,num_sal,num_rec,num_ser
0,0100100010229001,65,57,24,25,1,21,1,25,1,...,1.053104,0.324032,MICRO,C2-MICRO,"POLYGON ((2469592.842 1101420.419, 2469595.825...","POLYGON ((2469596.465 1101422.173, 2469599.209...",0,0,0,0
1,0100100010229002,0,0,0,0,0,0,0,0,0,...,6.803446,4.935834,MICRO,C2-MICRO,"POLYGON ((2469982.451 1100681.890, 2469958.510...","POLYGON ((2469984.567 1100678.488, 2469960.808...",0,0,0,0
2,0100100010229003,0,0,0,0,0,0,0,0,0,...,0.883456,0.110432,MEDIANA,C2-MEDIANA,"POLYGON ((2470515.420 1100233.447, 2470474.428...","POLYGON ((2470515.271 1100237.444, 2470515.371...",0,0,0,0
3,0100100010229004,0,0,0,0,0,0,0,0,0,...,3.957532,NaN,MICRO,C2-MICRO,"POLYGON ((2469819.355 1100035.603, 2469843.745...","POLYGON ((2469819.081 1100039.593, 2469843.462...",0,0,0,0
4,0100100010229006,25,26,8,10,1,9,1,10,1,...,NaN,8.405244,0,C2-0,"POLYGON ((2469559.613 1101504.720, 2469551.460...","POLYGON ((2469563.600 1101505.043, 2469563.606...",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200160,320560045,638,840,178,271,160,209,178,262,160,...,NaN,NaN,0,C2-0,"POLYGON ((2434525.558 1192261.698, 2434477.354...","POLYGON ((2434524.049 1192257.965, 2434476.289...",3,0,0,0
1200161,320560098,247,286,391,635,53,63,391,630,53,...,NaN,NaN,0,C2-0,"POLYGON ((2442700.998 1196775.421, 2442708.345...","POLYGON ((2442697.032 1196775.939, 2442697.048...",0,1,0,0
1200162,320170294,23,84,14,33,7,22,14,33,7,...,0.078551,NaN,MICRO,C2-MICRO,"POLYGON ((2450812.614 1192490.514, 2450801.553...","POLYGON ((2450814.718 1192493.916, 2450814.799...",0,0,0,0
1200163,320170036,667,597,167,215,135,149,164,213,135,...,NaN,NaN,0,C2-0,"POLYGON ((2446403.618 1185915.688, 2446329.967...","POLYGON ((2446407.434 1185914.490, 2446407.403...",3,0,0,1


In [ ]:
# Crear un buffer de 2 metros alrededor de las manzanas
gdf_manzanas['buffer_2m'] = gdf_manzanas.geometry.buffer(2)

# Establecer el buffer como la geometría activa
gdf_manzanas = gdf_manzanas.set_geometry('buffer_2m')

# Inicializar las columnas para contar equipamientos en cada categoría
gdf_manzanas['num_edu'] = 0
gdf_manzanas['num_sal'] = 0
gdf_manzanas['num_rec'] = 0
gdf_manzanas['num_ser'] = 0

# Lista de categorías y sus respectivas columnas de conteo
categorias = {
    'Educación': 'num_edu',
    'Salud': 'num_sal',
    'Recreación': 'num_rec',
    'Servicios': 'num_ser'
}

# Cargar cada capa de equipamientos desde el geopackage y contar los elementos por manzana
for categoria, columna_conteo in categorias.items():
    # Cargar la capa correspondiente desde el geopackage
    gdf_categoria = gpd.read_file(equipamientos_gpkg, layer=categoria)
    
    # Realizar un join espacial entre los buffers de manzanas y los equipamientos de la categoría
    joined = gpd.sjoin(gdf_manzanas[['buffer_2m']], gdf_categoria, how='left', op='contains')
    
    # Contar los equipamientos de la categoría por cada manzana
    gdf_manzanas[columna_conteo] = joined.groupby(joined.index)['index_right'].count()

    print(f"Conteo completado para la categoría: {categoria}")

In [10]:
#borrar columna del buffer
gdf_manzanas = gdf_manzanas.drop(columns=['buffer_2m'])

#Guardar la capa resultante en un archivo .gpkg
gdf_manzanas.to_file(manzanas_gpkg_path, layer='Mzn_SubTipologias_eq', driver="GPKG")

TypeError: Cannot interpret '<geopandas.array.GeometryDtype object at 0x000001B8D5B47F90>' as a data type

In [13]:
gdf_manzanas.columns

Index(['CVEGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020',
       'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
       'TVIVPARHAB2010',
       ...
       'DENS_EMP10', 'DENS_UEC20', 'DENS_UEC10', 'R_TAM', 'SUBTIPOLOGÍA_ue',
       'geometry', 'num_edu', 'num_sal', 'num_rec', 'num_ser'],
      dtype='object', length=135)

In [ ]:
#3.3: Aplicación de radios de influencia para los equipamientos

# Definir los valores de radio en metros para cada nivel de servicio
radio_niveles = {1: 1600, 2: 4800, 3: 14400}  # 1 km, 5 km, 12 km

# Crear los buffers basados en el nivel de servicio (campo 'RADIO')
gdf_categoria['buffer_influencia'] = gdf_categoria.apply(lambda row: row.geometry.buffer(radio_niveles[row['RADIO']]), axis=1)

# Establecer el buffer como la geometría activa
gdf_categoria = gdf_categoria.set_geometry('buffer_influencia')

#Guardar la capa en un archivo
gdf_categoria.to_file(equipamientos_gpkg, layer='Eq_Buffer_Influencia', driver="GPKG")

In [48]:
# 3.4: Sumar los valores de ponderación por manzana

# Inicializar las columnas de importancia por categoría
gdf_manzanas['IMP_EDU'] = 0
gdf_manzanas['IMP_SAL'] = 0
gdf_manzanas['IMP_REC'] = 0
gdf_manzanas['IMP_SER'] = 0

# Diccionario que asocia las categorías con las columnas de importancia
categorias = {
    'Educación': 'IMP_EDU',
    'Salud': 'IMP_SAL',
    'Recreación': 'IMP_REC',
    'Servicios': 'IMP_SER'
}

print(f'1 - {datetime.datetime.now().time()}')

# Dividir el GeoDataFrame de manzanas en partes más pequeñas
chunk_size = 20000  # Ajusta el tamaño según la memoria disponible
manzanas_chunks = np.array_split(gdf_manzanas, len(gdf_manzanas) // chunk_size)

# Procesar cada bloque de manzanas y sumarle la importancia de los equipamientos que intersectan
result_chunks = []

print(f'2 - {datetime.datetime.now().time()}')

for i, chunk in enumerate(manzanas_chunks):
    print(f"Procesando bloque {i+1} de {len(manzanas_chunks)}")

    # Copiar el chunk para no modificar el original
    chunk_copy = chunk.copy()
    
    for categoria, columna_importancia in categorias.items():
        
        # Join espacial entre el chunk de manzanas y los equipamientos por categoría usando el buffer de influencia
        joined = gpd.sjoin(
            chunk_copy, 
            gdf_categoria[['buffer_influencia', 'IMPORTANCIA']], 
            how='left', 
            predicate='intersects',
            lsuffix='mz', 
            rsuffix='eq'
        )
        
        # Sumar la importancia de los equipamientos que caen dentro de cada manzana
        sum_importancia = joined.groupby(joined.index)['IMPORTANCIA'].sum()
        
        # Actualizar la columna de importancia en el chunk original
        chunk_copy[columna_importancia] += chunk_copy.index.map(sum_importancia).fillna(0)

    # Añadir el chunk procesado a la lista de resultados
    result_chunks.append(chunk_copy)

print(f'3 - {datetime.datetime.now().time()}')

# Concatenar todos los chunks procesados para obtener el GeoDataFrame final
gdf_manzanas_final = pd.concat(result_chunks)

# Revisar el resultado
gdf_manzanas_final.head()


1 - 12:25:03.365873
2 - 12:41:20.397294
Procesando bloque 1 de 60


C:\Users\UTIC\AppData\Local\Temp\ipykernel_18572\2225949447.py:37: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:6372
Right CRS: None

  joined = gpd.sjoin(
C:\Users\UTIC\AppData\Local\Temp\ipykernel_18572\2225949447.py:37: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:6372
Right CRS: None

  joined = gpd.sjoin(


MemoryError: Unable to allocate 364. MiB for an array with shape (29, 1643660) and data type float64

In [ ]:
# 3.5: Visualización estadística de los valores de importancia por categoría

# Cargar el archivo de manzanas
gdf_manzanas = gpd.read_file(manzanas_gpkg_path, layer='Mzn_SubTipologias_eq')

# Resumen estadístico de las columnas de importancia
print(gdf_manzanas[['IMP_EDU', 'IMP_SAL', 'IMP_REC', 'IMP_SER']].describe())

# Visualización gráfica de los valores de importancia por categoría
gdf_manzanas[['IMP_EDU', 'IMP_SAL', 'IMP_REC', 'IMP_SER']].plot(kind='box', title='Importancia por Categoría')
plt.show()
